In [8]:
import torch
import torch.nn as nn
import numpy as np

In [3]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)

        last_hidden_state = lstm_out[:, -1, :]

        output = self.fc(last_hidden_state)
        return output
    def predict(self, x):
        '''
        Метод для консистентности работы бека
        '''
        self.eval()
        x = torch.Tensor(x)
        with torch.no_grad():
            y_hat = self.forward(x)
        return y_hat

In [19]:
tensor_uniform = torch.rand(3, 3)
tensor_uniform.unsqueeze(0).size()

torch.Size([1, 3, 3])

In [21]:
def get_error_metrics(y_val, y_hat) -> dict:
    '''
    Validates regression quality with R2-score and RMSE. 
    Please note that the method is suitable for either 
    1-step ahead and n-step ahead forecasts
    
    Returns a dict with results

    kwargs: y_val 
            y_hat

    returns: metrics_dict -> dict
    '''
    y_val = torch.Tensor(y_val.to_numpy()).to('cpu')
    y_hat = torch.Tensor(y_hat).to('cpu')
    y_val = y_val.cpu().numpy()
    y_hat = y_hat.cpu().numpy()
    _log = logging.getLogger(__name__)
    _log.info(f"Validating prediction quality...")

    metrics_dict = {"R2_score" : float(r2_score(y_true = y_val, y_pred = y_hat)),
                    "RMSE" : float(mean_squared_error(y_true = y_val, y_pred = y_hat, squared = False))}
    _log.info(f"Got offline metrics.")
    return metrics_dict

In [22]:
# Example usage
import numpy as np

# Creating random data for demonstration
X_train = np.random.randn(100, 10, 3)  # 100 samples, 10 time steps, 3 features
y_train = np.random.randn(100, 1)  # 100 samples, 1 target per sample

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Define model
input_size = 3
hidden_size = 4
output_size = 1
model = LSTMModel(input_size, hidden_size, output_size)

# Move model to device (CPU for simplicity)
device = torch.device('cpu')
model = model.to(device)

# Training loop (simplified)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()

# Making predictions
model.eval()
y_pred = model.predict(X_train_tensor)

# Getting error metrics
metrics = get_error_metrics(y_train_tensor, y_pred)
print(metrics)


AttributeError: 'Tensor' object has no attribute 'to_numpy'

In [24]:
y_pred.shape

torch.Size([100, 1])

In [25]:
import torch
import torch.nn as nn
import pandas as pd

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        last_hidden_state = lstm_out[:, -1, :]  # Take the last hidden state for each sequence
        output = self.fc(last_hidden_state)
        return output

    def predict(self, x):
        '''
        Method for consistency of backend operation
        '''
        self.eval()

        if isinstance(x, pd.DataFrame):
            x = torch.Tensor(x.to_numpy())

        # Ensure the input tensor has three dimensions [batch_size, sequence_length, input_size]
        if len(x.shape) == 2:
            x = x.unsqueeze(0)

        print(f'Input shape before sending to device: {x.shape}')
        device = next(self.parameters()).device
        x = x.to(device)
        print(f'Input shape after sending to device: {x.shape}')

        with torch.no_grad():
            y_hat = self.forward(x)

        print(f'Output shape from forward pass: {y_hat.shape}')

        return y_hat

# Example usage
import numpy as np

# Creating random data for demonstration
X_train = np.random.randn(67, 24, 3)  # 67 samples, 24 time steps, 3 features
y_train = np.random.randn(67, 1)  # 67 samples, 1 target per sample

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Define model
input_size = 3
hidden_size = 4
output_size = 1
model = LSTMModel(input_size, hidden_size, output_size)

# Move model to device (CPU for simplicity)
device = torch.device('cpu')
model = model.to(device)

# Training loop (simplified)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()

# Making predictions
model.eval()
y_pred = model.predict(X_train_tensor)

# Ensure y_pred and y_train_tensor have the same shape for metric calculation
y_pred = y_pred.cpu().numpy()
y_true = y_train_tensor.cpu().numpy()

# Getting error metrics
def get_error_metrics(y_true, y_pred):
    from sklearn.metrics import r2_score, mean_squared_error

    # Reshape if necessary
    if y_true.ndim > 1 and y_true.shape[1] == 1:
        y_true = y_true.squeeze()
    if y_pred.ndim > 1 and y_pred.shape[1] == 1:
        y_pred = y_pred.squeeze()

    metrics_dict = {
        "R2_score": float(r2_score(y_true=y_true, y_pred=y_pred)),
        "MSE": float(mean_squared_error(y_true=y_true, y_pred=y_pred))
    }
    return metrics_dict

metrics = get_error_metrics(y_true, y_pred)
print(metrics)


Input shape before sending to device: torch.Size([67, 24, 3])
Input shape after sending to device: torch.Size([67, 24, 3])
Output shape from forward pass: torch.Size([67, 1])


/Users/ab/Library/Caches/pypoetry/virtualenvs/ml-stock-prediction-project-WhoF3gG6-py3.9/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


{'R2_score': -0.10490214824676514, 'MSE': 1.2540651559829712}


In [12]:
LSTMModel_test = LSTMModel(1, 10, 1)

In [17]:
LSTMModel_test.predict([1])

AttributeError: 'list' object has no attribute 'dim'

In [13]:
LSTMModel_test.predict(torch.Tensor(np.arange(0,100)))

ValueError: LSTM: Expected input to be 2D or 3D, got 1D instead

In [16]:
torch.Tensor(np.arange(0,100)).shape

torch.Size([100])